# Data preparation/Chunking

In [2]:
!ls data

faq.json


In [3]:
import json
import os
import spacy
import faiss
from sentence_transformers import SentenceTransformer
import re

/Users/sthapa/Devs/rag/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sthapa/Devs/rag/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Chunking using: spacy and en_core_web_sm

In [4]:
nlp = spacy.load("en_core_web_sm")

def process_file(file_path):
    with open(file_path) as f:
        data = json.load(f)
        content = data["content"]
        url = data["url"]
        doc = nlp(content)

        return [{"text": sent.text, "url": url} for sent in doc.sents]

chunks = [
    chunk
    for file in os.listdir("data")
    for chunk in process_file(os.path.join("data", file))
]

chunks = [{"id": i, **chunk} for i, chunk in enumerate(chunks)]

with open("chunks.json", "w") as f:
    json.dump(chunks, f)

# Vector store

### vector embedding using: all_mpnet_base_v2 sentence_transformer

In [5]:
sentences = [chunk["text"] for chunk in chunks]

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

embeddings = model.encode(sentences, show_progress_bar=True)

Batches: 100%|██████████| 4/4 [00:00<00:00,  5.70it/s]


In [6]:
embeddings

array([[ 0.09477425, -0.01864169,  0.00911164, ..., -0.04637649,
         0.00228056,  0.01923566],
       [ 0.03039058, -0.03222821, -0.03873111, ..., -0.0035764 ,
         0.00579478, -0.0339357 ],
       [ 0.05490492, -0.06632555,  0.0165215 , ..., -0.02763968,
        -0.00135091, -0.00359141],
       ...,
       [ 0.02805713, -0.06807668, -0.02322498, ..., -0.0072787 ,
        -0.00197908,  0.00630819],
       [-0.0140816 ,  0.02815344,  0.00403775, ...,  0.03194111,
        -0.01472967,  0.02749774],
       [ 0.01043997,  0.01733964, -0.02638827, ..., -0.0227873 ,
        -0.03920399, -0.04961954]], dtype=float32)

In [7]:
faiss_index = faiss.IndexFlatIP(model.get_sentence_embedding_dimension())
faiss_index.add(embeddings)

# Retrieval/Prompt preparation

In [8]:
base_prompt = """You are an AI assistant. Your task is to understand the user question, and provide an answer using the provided contexts. Every answer you generate should have citations in this pattern  "Answer [position].", for example: "Earth is round [1][2].," if it's relevant.

Your answers are correct, high-quality, and written by an domain expert. If the provided context does not contain the answer, simply state, "The provided contexts does not have the answer."

User question: {}

Contexts:
{}
"""

In [9]:
k = 50
question = "Why HackerNews is so popular?"

query_embedding = model.encode([question])
distances, indices = faiss_index.search(query_embedding, k)

context = "\n".join([f"{i}. {sentences[index]}" for i, index in enumerate(indices[0])])
prompt = f"{base_prompt.format(question, context)}"

In [10]:
print(context)
print("--------------------------------")
print(prompt)

0. Hacker News FAQ

Are there rules about submissions and comments?https://news.ycombinator.com/newsguidelines.html
How are stories ranked?

1. HN gives three features to YC: job ads (see above) and startup launches get placed on the front page, and YC founder names are displayed to other YC alumni in orange.

2. It's a way to help you prevent yourself from spending too much time on HN.
3. Users should vote for a story because they personally find it intellectually interesting, not because someone has content to promote.
4. We don't want anyone to get in trouble from anything they posted to HN.
5. Why is A ranked below B even though A has more points and is newer?

6. These appear on the front page, but are not stories: they have no vote arrows, points, or comments.
7. Other factors affecting rank include user flags, anti-abuse software, software which demotes overheated discussions, account or site weighting, and moderator action.

8. http://news.ycombinator.com/formatdoc

9. Some vot

# Answer Generation

In [12]:
import openai
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key = "sk-deepseek-dummy-key",    
)

try:
    response = client.chat.completions.create(
        model="deepseek-r1:8b",
        temperature=0.7,
        messages=[
            {"role": "system", "content": "You are a helpful and informative assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    print(response.choices[0].message.content)
except openai.OpenAIError as e:
    print(f"An error occurred: {e}")
    print("\nPlease ensure your Ollama server is running and the model is downloaded.")


<think>
Okay, I'm trying to figure out why Hacker News is so popular. Let me start by looking at the contexts provided.

From context 0, it mentions that Hacker News has rules about submissions and comments, including how stories are ranked. Context 1 says that Y Combinator (YC) gives three features to YC: job ads, startup launches, and founder names displayed in orange. So maybe the connection with YC is a big part.

Context 2 suggests it's a way to help users avoid spending too much time on HN, which implies it's useful but can be addictive. Context 3 says users should vote for stories because they find them intellectually interesting, not just to promote something. That makes me think about the community-driven aspect and quality content.

Contexts 4 and 49 talk about privacy and protecting individual users, so maybe HN is seen as a trustworthy place. Context 5 asks why one story is ranked below another even if it has more points and is newer, which shows there's an algorithm but it

Check OPEN AI using Deepseek R1

In [ ]:
import openai
import os

# --- New syntax for openai library v1.0.0+ ---
# Import the OpenAI client`` class
from openai import OpenAI

# Create an instance of the OpenAI client.
# This client object will handle the API calls.
# Set the base_url to the correct HTTP endpoint for Ollama.
# Note that it's 'http' not 'https'
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key = "sk-deepseek-dummy-key",    
)

# Define a sample prompt for the model.
prompt = "Explain the concept of 'lazy evaluation' in programming."

try:
    # Use the client instance to make the API call with the new syntax.
    # The method is now `client.chat.completions.create`.
    # The arguments are largely the same.
    response = client.chat.completions.create(
        model="deepseek-r1:8b",
        temperature=0.7,
        messages=[
            {"role": "system", "content": "You are a helpful and informative assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Access the content from the response object. The syntax for this also changed
    # from a dictionary-like access to a more object-oriented one.
    print(response.choices[0].message.content)

except openai.OpenAIError as e:
    # The exception class has changed, but you can still catch it.
    print(f"An error occurred: {e}")
    print("\nPlease ensure your Ollama server is running and the model is downloaded.")


<think>
Okay, so I'm trying to understand what "lazy evaluation" means in programming. I've heard the term before, maybe in the context of functional programming or something like that. But I'm not entirely sure about it. Let me try to break it down.

First, I think evaluation has to do with how expressions are processed by a computer. So when we say "lazy evaluation," does that mean something is being evaluated later than expected? Maybe it's the opposite of eager evaluation. Hmm, I remember hearing that in some languages, like Haskell or Lisp, functions aren't evaluated immediately but when needed.

So, if I have a function that depends on another function, under lazy evaluation, the second function isn't computed until the first one actually needs its result. That makes sense because it could save computation time if the result isn't needed right away. For example, in a list comprehension or something where each element is processed conditionally.

I think this approach helps with e